In [1]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

#Busca o arquivo com as medições de KPIs de Dia Médio
os.chdir("C:/Users/f8026379/Documents/MANP/Operating_Point")
data1=pd.read_excel('Base_Dimensionamento_C1_decong..xlsx', sheet_name='Sheet1')

#Calcula medias moveis e gera gráfficos
os.chdir("C:/Users/f8026379/Documents/MANP/Operating_Point/Graphics")
data2=data1[['Fornecedor','Banda','BW','User Thp','Utilization']]
result=DataFrame([])
for (f,b,bw) , group in data2.groupby(['Fornecedor','Banda','BW']):
    test=group
    test=test.reset_index(drop=True)
    if test.count()[0] >100:
        w=int((test.count()[0])/100)
    else:
        w=2
    Mov_avg_input= test[['User Thp','Utilization']]
    Mov_avg_input=Mov_avg_input.sort_values(by=['Utilization'])
    Mov_avg_output=Mov_avg_input.rolling(window=w).mean().dropna()
    Mov_avg_output[['Fornecedor','Banda','BW']]=test[['Fornecedor','Banda','BW']]
    result=pd.concat([result,Mov_avg_output],sort=False)
    plt.close()
    plt.scatter(test['Utilization']*100,test['User Thp'],color='cyan')
    plt.plot(Mov_avg_output['Utilization']*100,Mov_avg_output['User Thp'],color='black')
    plt.title(f"U_Thp x Util \nVendor: {f!r} Layer: {b!r} BW: {bw!r} MHz")
    plt.axhline(linewidth=1, color='y',y=5000)
    plt.axhline(linewidth=1, color='r',y=3000)
    plt.grid(linestyle='--')
    plt.xlabel('Utilization [%]')
    plt.ylabel("U_Thp [Kbps]")
    plt.xlim(left=0)
    plt.xlim(right=100)
    plt.ylim(top=35000)
    plt.ylim(bottom=0)
    plt.text(20,3000,"Crítico")
    plt.text(20,5000,"Alarme")
    #plt.autoscale(enable=True)
    plt.savefig('U_Thp_x_Util_{}'.format(f+"_"+b+"_"+str(bw)+" MHz").format(bw))

#Geração dos gráficos comparativos por Fornecedores
os.chdir("C:/Users/f8026379/Documents/MANP/Operating_Point/Graphics_Vendors")
f_ser=Series(data1.Fornecedor.unique())
bw_ser=Series(data1.BW.unique())
b_ser=Series(data1.Banda.unique())
for j in b_ser.index:
        b=b_ser[j]
        for k in bw_ser.index:
            bw=bw_ser[k]
            plt.close()
            Ericsson=result[result['Fornecedor'].isin(['ERICSSON'])&result['BW'].isin([bw])&result['Banda'].isin([b])]
            Nokia=result[result['Fornecedor'].isin(['NOKIA'])&result['BW'].isin([bw])&result['Banda'].isin([b])]
            Huawei=result[result['Fornecedor'].isin(['HUAWEI'])&result['BW'].isin([bw])&result['Banda'].isin([b])]
            if Nokia.count()[1]+Ericsson.count()[1]+Huawei.count()[1]>0:
                Ericsson=Ericsson.reset_index(drop=True)
                Ericsson[['ERI']]=Ericsson[['User Thp']]
                plt.plot(Ericsson['Utilization']*100,Ericsson['ERI'])            
                Nokia=Nokia.reset_index(drop=True)
                Nokia[['NSN']]=Nokia[['User Thp']]
                plt.plot(Nokia['Utilization']*100,Nokia['NSN'])            
                Huawei=Huawei.reset_index(drop=True)
                Huawei[['HUA']]=Huawei[['User Thp']]
                plt.plot(Huawei['Utilization']*100,Huawei['HUA'])    
                plt.title(f"U_Thp x Util \n Layer: {b!r} BW: {bw!r} MHz")
                plt.axhline(linewidth=1, color='y',y=5000)
                plt.axhline(linewidth=1, color='r',y=3000)
                plt.grid(linestyle='--')
                plt.text(0,3000,"Crítico")
                plt.text(0,5000,"Alarme")
                plt.xlim(right=0)
                plt.xlim(right=100)
                plt.ylim(bottom=0)
                plt.xlabel('Utilization [%]')
                plt.ylabel("U_Thp [Kbps]")
                plt.legend()
                plt.savefig('Vendors_{}'.format(b+"_"+str(bw)+" MHz").format(bw))
    
    
#Geração dos Reports
os.chdir("C:/Users/f8026379/Documents/MANP/Operating_Point/Reports")
result_5M=result[np.logical_and(result['User Thp']>4500,result['User Thp']<5500)]
result_3M=result[np.logical_and(result['User Thp']>2700,result['User Thp']<3300)]
Samples=result.pivot_table(columns=['Fornecedor'],index=['Banda','BW'],values=['User Thp'],aggfunc='count')
pivot_5m=result_5M.pivot_table(columns=['Fornecedor'],index=['Banda','BW'],values=['Utilization'])
pivot_3m=result_3M.pivot_table(columns=['Fornecedor'],index=['Banda','BW'],values=['Utilization'])
# Specify a writer
writer = pd.ExcelWriter('Report.xlsx', engine='xlsxwriter')
# Write your DataFrame to a file     
result.to_excel(writer, 'Mov_average_total')
Samples.to_excel(writer, 'Mov_average_total',startcol=10)
pivot_3m.to_excel(writer, 'Op_points_3Mbps')
pivot_5m.to_excel(writer, 'Op_points_5Mbps')
result_3M.to_excel(writer, 'Samples_3Mbps')
result_5M.to_excel(writer, 'Samples_5Mbps')
# Save the result 
writer.save()

In [60]:
def mov_avg(df):
    df=df.rolling(window=100).mean().dropna()
    return df

In [61]:
test=data2.groupby(['Fornecedor','Banda','BW']).apply(mov_avg)

In [38]:
# The normal imports
import os
import numpy as np
from numpy.random import randn
import pandas as pd

# Import the stats library from numpy
from scipy import stats

# These are the plotting modules adn libraries we'll use:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Command so that plots appear in the iPython Notebook
%matplotlib inline

#Busca o arquivo com as medições de KPIs de Dia Médio
os.chdir("C:/Users/f8026379/Documents/MANP/Operating_Point")
data1=pd.read_excel('Base_Dimensionamento_C1_decong..xlsx', sheet_name='Sheet1')

data2=data1[['Fornecedor','Banda','BW','User Thp','Utilization']]

In [40]:
data2.head()

,Fornecedor,Banda,BW,User Thp,Utilization
0,ERICSSON,L2600,10,19046.183088,0.319697
1,ERICSSON,L2600,10,23116.819849,0.063849
2,NOKIA,L700,10,1352.060909,2.930659
3,NOKIA,L700,10,1447.699725,2.795563
4,NOKIA,L700,10,1790.847798,2.761834


In [7]:
#sns.lmplot("size", "tip_pect", tips, x_estimator=np.mean)

In [48]:
a=pd.cut(data2.Utilization,10)

In [49]:
a

0        (-0.00351, 0.352]
1        (-0.00351, 0.352]
2           (2.816, 3.168]
3           (2.464, 2.816]
4           (2.464, 2.816]
5        (-0.00351, 0.352]
6           (0.352, 0.704]
7        (-0.00351, 0.352]
8           (0.352, 0.704]
9           (0.352, 0.704]
10          (0.352, 0.704]
11          (0.352, 0.704]
12          (0.352, 0.704]
13       (-0.00351, 0.352]
14       (-0.00351, 0.352]
15       (-0.00351, 0.352]
16       (-0.00351, 0.352]
17       (-0.00351, 0.352]
18          (0.704, 1.056]
19          (0.352, 0.704]
20           (1.408, 1.76]
21          (0.704, 1.056]
22          (0.352, 0.704]
23          (0.352, 0.704]
24          (0.352, 0.704]
25       (-0.00351, 0.352]
26       (-0.00351, 0.352]
27       (-0.00351, 0.352]
28       (-0.00351, 0.352]
29          (0.352, 0.704]
               ...        
86514       (0.352, 0.704]
86515       (0.352, 0.704]
86516       (0.352, 0.704]
86517    (-0.00351, 0.352]
86518    (-0.00351, 0.352]
86519    (-0.00351, 0.352]
8

In [50]:
a=pd.DataFrame(a)

In [52]:
a.tail()


,Utilization
86539,"(-0.00351, 0.352]"
86540,"(-0.00351, 0.352]"
86541,"(-0.00351, 0.352]"
86542,"(-0.00351, 0.352]"
86543,"(-0.00351, 0.352]"


In [53]:
data2.tail()

,Fornecedor,Banda,BW,User Thp,Utilization
86539,HUAWEI,L2600,10,41284.318155,0.341763
86540,HUAWEI,L2600,10,10770.706216,0.238737
86541,HUAWEI,L2600,10,6983.875269,0.177005
86542,HUAWEI,L2600,10,5696.789887,0.309728
86543,HUAWEI,L2600,10,12640.163892,0.131616


In [ ]:
test=pd.merge(data)